In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

In [2]:
#载入数据集
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)
#运行次数
max_steps=1001
#图片数量
image_num=3000

#定义绘画
sess=tf.Session()

#载入图片
embedding=tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False,name='embedding')

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean=tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)#平均值,记录这个值，并给一个名字
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)#标准差
        tf.summary.scalar('max',tf.reduce_max(var))#最大值
        tf.summary.scalar('min',tf.reduce_min(var))#最小值
        tf.summary.histogram('histongram',var)#直方图
#命名空间
with tf.name_scope('input'):
    #定义两个占位符
    x=tf.placeholder(tf.float32,[None,784],name='x_input')
    y=tf.placeholder(tf.float32,[None,10],name='y_input')

#显示图片
with tf.name_scope('input_reshape'):
    image_shaped_input=tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input',image_shaped_input,10)#放10张图片
    
#创建一个简单的神经网络
with tf.name_scope('layer'):
    with tf.name_scope('withts'):
        W=tf.Variable(tf.zeros([784,10]),name='w')
        variable_summaries(W)
    with tf.name_scope('biases'):
        b=tf.Variable(tf.zeros([10]),name='b')
        variable_summaries(b)
    with tf.name_scope('wx_plis_b'):
        wx_plus_b=tf.matmul(x,W)+b
    with tf.name_scope('softmax'):
        prediction=tf.nn.softmax(wx_plus_b)

#二次代价函数
# loss=tf.reduce_mean(tf.square(y-prediction))
with tf.name_scope('loss'):
    #交叉熵？？？？加快收敛速度
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)#loss只有一个值，没有标准差
with tf.name_scope('train'):
    train_step=tf.train.GradientDescentOptimizer(0.2).minimize(loss)
init=tf.global_variables_initializer()
sess.run(init)
with tf.name_scope('accuracy'):
#求准确率,比较两个参数是否相同，相同返回True，不同返回False。
    with tf.name_scope('correct_prediction'):
        correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大值所在的位置
    with tf.name_scope('accuracy'):
        accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))#cast将布尔转化为浮点
        tf.summary.scalar('accuracy',accuracy)

#产生metadata文件
if tf.gfile.Exists('projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively('projector/projector/metadata.tsv')
with open('projector/projector/metadata.tsv','w') as f:
    labels=sess.run(tf.argmax(mnist.test.labels[:],1))
    for i in range(image_num):
        f.write(str(labels[i])+'\n')
#合并所有summary
merged=tf.summary.merge_all()


projector_writer=tf.summary.FileWriter('projector/projector',sess.graph)#文件路径
saver=tf.train.Saver()
config=projector.ProjectorConfig()
embed=config.embeddings.add()
embed.tensor_name=embedding.name
embed.metadata_path='projector/projector/metadata.tsv'
embed.sprite.image_path='projector/data/mnist_10k_sprite.png'
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer,config)

for i in range(max_steps):
    batch_xs,batch_ys=mnist.train.next_batch(100)
    run_options=tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata=tf.RunMetadata()
    summary,_=sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata,'step%03d'%i)
    projector_writer.add_summary(summary,i)
    
    if i%100==0:
        acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter"+str(i)+", Testing Accuracy="+str(acc))

        
saver.save(sess,'projector/projector/a_model.ckpt',global_step=max_steps)
projector_writer.close()
sess.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter0, Testing Accuracy=0.1552
Iter100, Testing Accuracy=0.7542
Iter200, Testing Accuracy=0.7993
Iter300, Testing Accuracy=0.8122
Iter400, Testing Accuracy=0.8179
Iter500, Testing Accuracy=0.8229
Iter600, Testing Accuracy=0.8241
Iter700, Testing Accuracy=0.8282
Iter800, Testing Accuracy=0.832
Iter900, Testing Accuracy=0.8613
Iter1000, Testing Accuracy=0.8762


In [3]:
help(tf.gfile.DeleteRecursively)

Help on function delete_recursively in module tensorflow.python.lib.io.file_io:

delete_recursively(dirname)
    Deletes everything under dirname recursively.
    
    Args:
      dirname: string, a path to a directory
    
    Raises:
      errors.OpError: If the operation fails.

